## **Transformar datos para obtener ranking según el team**

*   Trabajamos con el resultado obtenido en el notebook **race_results**

In [ ]:
# Llamamos al notebook que contiene las variables de configuración
%run "../utils/configuration"

### Paso 1 - Leer **race_results** de la capa **presentation**

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc
from pyspark.sql.functions import sum, when, count, col

In [ ]:
# El parámetro "presentation_folder_path" se encuentra en el notebook "configuration"
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results")

### Paso 2 - Aplicar una agrupación y agregación a **race_results**

In [ ]:
# Cuando posicion es igual a 1 (es decir, el piloto ha ganado la carrera), entonces, es igual a 1 (True = 1 y False = 0)
constructor_standings_df = race_results_df \
.groupBy("race_year", "team") \
.agg(sum("points").alias("total_points"),
     count(when(col("position") == 1, True)).alias("wins")).sort(col('wins').desc()) 

### Paso 3 - Al resultado previo aplicar una ventana y calcular el ranking

In [ ]:
constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = constructor_standings_df.withColumn("rank", rank().over(constructor_rank_spec))

### Paso 4 - Escribir datos en el datalake como **parquet** y crear la tabla **constructor_standings** en la base de datos **f1_presentation**

In [ ]:
# Escribimos el archivo con formato PARQUET en la base de datos "f1_presentation" y en la tabla "constructor_standings"
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")